# Pinecone VectorStore와 HybridSearch 구현

Pinecone은 고성능 벡터 데이터베이스로, AI 및 머신러닝 애플리케이션을 위한 효율적인 벡터 저장 및 검색 솔루션입니다.

이 노트북에서는 Pinecone을 활용한 하이브리드 검색(Dense + Sparse Vector) 구현 방법을 학습합니다.

## Pinecone vs Chroma vs Faiss 비교

### Pinecone의 장점

1. **확장성**: 대규모 데이터셋에 대해 뛰어난 확장성을 제공합니다.
2. **관리 용이성**: 완전 관리형 서비스로, 인프라 관리 부담이 적습니다.
3. **실시간 업데이트**: 데이터의 실시간 삽입, 업데이트, 삭제가 가능합니다.
4. **고가용성**: 클라우드 기반으로 높은 가용성과 내구성을 제공합니다.
5. **API 친화적**: RESTful/Python API를 통해 쉽게 통합할 수 있습니다.

### Pinecone의 단점

1. **비용**: Chroma나 Faiss에 비해 상대적으로 비용이 높을 수 있습니다.
2. **커스터마이징 제한**: 완전 관리형 서비스이기 때문에 세부적인 커스터마이징에 제한이 있을 수 있습니다.
3. **데이터 위치**: 클라우드에 데이터를 저장해야 하므로, 데이터 주권 문제가 있을 수 있습니다.

### 비교 요약

- **Chroma/FAISS**: 오픈소스이며 로컬에서 실행 가능하여 초기 비용이 낮고 데이터 제어가 용이합니다. 커스터마이징의 자유도가 높습니다. 하지만 대규모 확장성 면에서는 Pinecone에 비해 제한적일 수 있습니다.

- **선택 기준**: 프로젝트의 규모, 요구사항, 예산 등을 고려하여 결정해야 합니다. 대규모 프로덕션 환경에서는 Pinecone이 유리할 수 있지만, 소규모 프로젝트나 실험적인 환경에서는 Chroma나 Faiss가 더 적합할 수 있습니다.

**참고 자료**
- [Pinecone 공식 홈페이지](https://docs.pinecone.io/integrations/langchain)
- [Pinecone 랭체인 통합](https://python.langchain.com/v0.2/docs/integrations/vectorstores/pinecone/)

## 환경 설정

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

## 한글 처리를 위한 불용어 사전

한글 텍스트 처리를 위해 불용어(stopwords) 사전을 준비합니다. 이는 추후 토크나이저에서 사용됩니다.

In [2]:
from langchain_teddynote.korean import stopwords

# 한글 불용어 사전 불러오기 (불용어 사전 출처: https://www.ranks.nl/stopwords/korean)
stopword = stopwords()
stopword[:20]

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다']

## 데이터 전처리

PDF 문서를 로드하고 적절한 크기로 분할합니다.

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import glob

# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)

split_docs = []

# PDF 파일들을 로드하고 분할
files = sorted(glob.glob("data/*.pdf"))

for file in files:
    loader = PyMuPDFLoader(file)
    split_docs.extend(loader.load_and_split(text_splitter))

# 문서 개수 확인
len(split_docs)

4

In [4]:
# 첫 번째 문서의 내용 확인
split_docs[0].page_content

'<ADsP 요약정리 및 오답노트>\n-1과목-\n(객관식)\n데이터 마스킹 : 데이터의 속성은 유지한 채, 익명으로 생성\nCinematch -> 넷플릭스에서 개발한 알고리즘\n데이터마이닝 vs 머신러닝(딥러닝) 구분하기 다른거임\n트레이딩, 공급, 수요예측 -> 에너지 산업\nCRM -> 고객관계관리 데이터베이스 (기업내부)\nERP -> 기업 전체를 통합적으로 관리하고 경영의 효율화 목적\n빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발'

In [5]:
# 메타데이터 확인
split_docs[0].metadata

{'source': 'data\\ADsP.pdf',
 'file_path': 'data\\ADsP.pdf',
 'page': 0,
 'total_pages': 1,
 'format': 'PDF 1.4',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'PyPDF2',
 'creationDate': '',
 'modDate': '',
 'trapped': ''}

### 문서의 전처리

Pinecone에 저장하기 위해 문서를 전처리합니다.

- 필요한 `metadata` 정보를 추출합니다.
- 최소 길이 이상의 데이터만 필터링합니다.
- 문서의 `basename`을 사용할지 여부를 지정합니다. (파일 경로의 마지막 부분만 사용)

In [6]:
from langchain_teddynote.community.pinecone import preprocess_documents

# 문서 전처리: 메타데이터 추출 및 필터링
contents, metadatas = preprocess_documents(
    split_docs=split_docs,
    metadata_keys=["source", "page", "author"],  # 저장할 메타데이터 키
    min_length=5,  # 최소 문서 길이
    use_basename=True,  # 파일명만 사용
)

  0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
# 전처리 결과 확인
print(f"문서 개수: {len(contents)}")
print(f"메타데이터 키: {metadatas.keys()}")
print(f"소스 예시: {metadatas['source'][:5]}")

문서 개수: 4
메타데이터 키: dict_keys(['source', 'page', 'author'])
소스 예시: ['ADsP.pdf', 'ADsP.pdf', 'ADsP.pdf', 'ADsP.pdf']


## Pinecone API 키 설정

Pinecone을 사용하려면 API 키가 필요합니다.

### API 키 발급 방법
1. [Pinecone 웹사이트](https://app.pinecone.io/) 접속
2. 프로필 → Account → Projects → Starter → API keys → 발급
3. `.env` 파일에 다음과 같이 추가:
   ```
   PINECONE_API_KEY="YOUR_PINECONE_API_KEY"
   ```

## 새로운 VectorStore 인덱스 생성

Pinecone의 새로운 인덱스를 생성합니다.

**주의사항**
- `metric`은 유사도 측정 방법을 지정합니다. HybridSearch를 사용할 경우 `dotproduct`로 설정해야 합니다.
- `dimension`은 사용하는 임베딩 모델의 차원과 일치해야 합니다.

In [8]:
import os
from langchain_teddynote.community.pinecone import create_index

# Pinecone 인덱스 생성 (무료 Serverless)
pc_index = create_index(
    api_key=os.environ["PINECONE_API_KEY"],
    index_name="db-index",  # 인덱스 이름
    dimension=4096,  # Embedding 차원 (OpenAI: 1536, Upstage: 4096)
    metric="dotproduct",  # 유사도 측정 방법 (dotproduct, euclidean, cosine)
)

[create_index]
{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}


### 유료 Pod 사용 예시 (선택사항)

유료 Pod는 무료 Serverless Pod 대비 더 확장된 기능을 제공합니다.

```python
# 유료 Pod 사용 예시
from pinecone import PodSpec

pc_index = create_index(
    api_key=os.environ["PINECONE_API_KEY"],
    index_name="teddynote-db-index2",
    dimension=4096,
    metric="dotproduct",
    pod_spec=PodSpec(
        environment="us-west1-gcp", 
        pod_type="p1.x1", 
        pods=1
    ),
)
```

참고: https://docs.pinecone.io/guides/indexes/choose-a-pod-type-and-size

## Sparse Encoder 생성

HybridSearch를 위해 Sparse Encoder를 생성합니다.
- Kiwi 형태소 분석기와 한글 불용어 처리를 수행합니다.
- BM25 알고리즘을 사용하여 Sparse Vector를 생성합니다.

In [9]:
from langchain_teddynote.community.pinecone import (
    create_sparse_encoder,
    fit_sparse_encoder,
)

# 한글 불용어 사전 + Kiwi 형태소 분석기를 사용하여 Sparse Encoder 생성
sparse_encoder = create_sparse_encoder(stopwords(), mode="kiwi")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


### Sparse Encoder 학습

생성한 Sparse Encoder에 문서 corpus를 학습시킵니다.
학습된 인코더는 파일로 저장되어 나중에 쿼리 임베딩 시 사용됩니다.

In [10]:
# Sparse Encoder를 사용하여 contents를 학습
saved_path = fit_sparse_encoder(
    sparse_encoder=sparse_encoder, 
    contents=contents, 
    save_path="./sparse_encoder.pkl"  # 인코더 저장 경로
)

  0%|          | 0/4 [00:00<?, ?it/s]

[fit_sparse_encoder]
Saved Sparse Encoder to: ./sparse_encoder.pkl


## 임베딩 모델 설정

Dense Vector를 생성하기 위한 임베딩 모델을 설정합니다.

In [11]:
from langchain_openai import OpenAIEmbeddings
from langchain_upstage import UpstageEmbeddings

# 임베딩 모델 생성 (필요에 따라 선택)
openai_embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
upstage_embeddings = UpstageEmbeddings(model="solar-embedding-1-large-passage")

## Pinecone 인덱스에 문서 업로드 (Upsert)

전처리된 문서를 Pinecone 인덱스에 업로드합니다.
각 문서는 다음 정보를 포함합니다:
- `context`: 문서의 내용
- `metadata`: 페이지 번호, 출처 등의 메타데이터
- `values`: Dense Embedding (임베딩 모델을 통해 생성)
- `sparse_values`: Sparse Embedding (BM25를 통해 생성)

In [12]:
%%time
from langchain_teddynote.community.pinecone import upsert_documents

# 문서를 Pinecone에 업로드
upsert_documents(
    index=pc_index,  # Pinecone 인덱스
    namespace="teddynote-namespace-01",  # Pinecone namespace
    contents=contents,  # 전처리한 문서 내용
    metadatas=metadatas,  # 전처리한 문서 메타데이터
    sparse_encoder=sparse_encoder,  # Sparse encoder
    embedder=upstage_embeddings,  # Dense embedder
    batch_size=32,  # 배치 크기
)

  0%|          | 0/1 [00:00<?, ?it/s]

[upsert_documents]
{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'teddynote-namespace-01': {'vector_count': 0}},
 'total_vector_count': 0}
CPU times: total: 93.8 ms
Wall time: 2.75 s


### 대용량 문서 병렬 업로드 (선택사항)

대용량 문서를 빠르게 업로드하려면 병렬 처리를 사용할 수 있습니다.

```python
from langchain_teddynote.community.pinecone import upsert_documents_parallel

upsert_documents_parallel(
    index=pc_index,
    namespace="teddynote-namespace-02",
    contents=contents,
    metadatas=metadatas,
    sparse_encoder=sparse_encoder,
    embedder=upstage_embeddings,
    batch_size=64,
    max_workers=30,  # 병렬 처리 워커 수
)
```

## 인덱스 조회 및 관리

`describe_index_stats` 메서드로 인덱스의 통계 정보를 확인할 수 있습니다.

In [13]:
# 인덱스 통계 조회
pc_index.describe_index_stats()

{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'teddynote-namespace-01': {'vector_count': 4}},
 'total_vector_count': 4}

### 네임스페이스 삭제

필요한 경우 특정 네임스페이스의 모든 데이터를 삭제할 수 있습니다.

In [14]:
from langchain_teddynote.community.pinecone import delete_namespace

# 네임스페이스 삭제 (필요시)
# delete_namespace(
#     pinecone_index=pc_index,
#     namespace="teddynote-namespace-01",
# )

## 검색기(Retriever) 생성

### PineconeKiwiHybridRetriever 초기화

`PineconeKiwiHybridRetriever`는 Dense Vector와 Sparse Vector를 결합한 하이브리드 검색을 수행합니다.

**주요 파라미터**
- `index_name`: Pinecone 인덱스 이름
- `namespace`: 사용할 네임스페이스
- `api_key`: Pinecone API 키
- `sparse_encoder_path`: 저장된 Sparse Encoder 파일 경로
- `stopwords`: 불용어 리스트
- `tokenizer`: 사용할 토크나이저 (기본값: "kiwi")
- `embeddings`: Dense Embedding 모델
- `top_k`: 반환할 최대 문서 수
- `alpha`: Dense와 Sparse 벡터의 가중치 (0~1, 1에 가까울수록 Dense 가중치 높음)

In [15]:
import os
from langchain_teddynote.korean import stopwords
from langchain_teddynote.community.pinecone import init_pinecone_index
from langchain_upstage import UpstageEmbeddings

# Pinecone 검색기 파라미터 초기화
pinecone_params = init_pinecone_index(
    index_name="db-index",  # Pinecone 인덱스 이름
    namespace="teddynote-namespace-01",  # Pinecone Namespace
    api_key=os.environ["PINECONE_API_KEY"],  # Pinecone API Key
    sparse_encoder_path="./sparse_encoder.pkl",  # Sparse Encoder 저장경로
    stopwords=stopwords(),  # 불용어 사전
    tokenizer="kiwi",  # 한글 토크나이저
    embeddings=UpstageEmbeddings(
        model="solar-embedding-1-large-query"  # Query용 임베딩 모델
    ),
    top_k=5,  # Top-K 문서 반환 개수
    alpha=0.5,  # alpha=0.75인 경우: Dense 75%, Sparse 25%
)

[init_pinecone_index]
{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'teddynote-namespace-01': {'vector_count': 0}},
 'total_vector_count': 0}


### PineconeKiwiHybridRetriever 생성

초기화된 파라미터를 사용하여 하이브리드 검색기를 생성합니다.

In [16]:
from langchain_teddynote.community.pinecone import PineconeKiwiHybridRetriever

# 검색기 생성
pinecone_retriever = PineconeKiwiHybridRetriever(**pinecone_params)

## 검색 수행

생성한 검색기를 사용하여 다양한 검색을 수행할 수 있습니다.

In [17]:
# 기본 검색 수행
search_results = pinecone_retriever.invoke("데이터 마스킹 관련 정보에 대해서 알려줘")

# 검색 결과 출력
for i, result in enumerate(search_results, 1):
    print(f"=== 문서 {i} ===")
    print(f"내용: {result.page_content}")
    print(f"\n메타데이터: {result.metadata}")
    print("\n====================\n")

=== 문서 1 ===
내용: <ADsP 요약정리 및 오답노트>
-1과목-
(객관식)
데이터 마스킹 : 데이터의 속성은 유지한 채, 익명으로 생성
Cinematch -> 넷플릭스에서 개발한 알고리즘
데이터마이닝 vs 머신러닝(딥러닝) 구분하기 다른거임
트레이딩, 공급, 수요예측 -> 에너지 산업
CRM -> 고객관계관리 데이터베이스 (기업내부)
ERP -> 기업 전체를 통합적으로 관리하고 경영의 효율화 목적
빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발

메타데이터: {'author': '', 'context': '<ADsP 요약정리 및 오답노트>\n-1과목-\n(객관식)\n데이터 마스킹 : 데이터의 속성은 유지한 채, 익명으로 생성\nCinematch -> 넷플릭스에서 개발한 알고리즘\n데이터마이닝 vs 머신러닝(딥러닝) 구분하기 다른거임\n트레이딩, 공급, 수요예측 -> 에너지 산업\nCRM -> 고객관계관리 데이터베이스 (기업내부)\nERP -> 기업 전체를 통합적으로 관리하고 경영의 효율화 목적\n빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발', 'page': 0.0, 'source': 'ADsP.pdf'}


=== 문서 2 ===
내용: 빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발
                                          2) 새로운 가치 창출
                                          3) 분석 기술 발전
cf. 전문인력증가는 가치측정과 관련 없음
사생활 침해를 막기 위한 개인정보 무작위 처리 (본래 목적 외에 사용 방지기술) -> 난수화
유형분석 -> 특성에따라 분류할때 사용한다.
핀테크(금융)분야에서 빅데이터 활용의 핵심분야 -> 신용평가

메타데이터: {'author': '', 'context': '빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발\n         

### 동적 검색 파라미터 사용

`search_kwargs`를 사용하여 검색 시 동적으로 파라미터를 조정할 수 있습니다.

In [18]:
# k 파라미터를 사용하여 반환 문서 수 제한
search_results = pinecone_retriever.invoke(
    "데이터 관련 정보에 대해서 알려줘", 
    search_kwargs={"k": 1}  # 1개 문서만 반환
)
print(f"검색된 문서 수: {len(search_results)}")

검색된 문서 수: 1


In [19]:
# alpha 파라미터를 사용하여 Dense/Sparse 가중치 조정
# alpha=1: Dense Vector만 사용
# alpha=0: Sparse Vector만 사용
search_results = pinecone_retriever.invoke(
    "데이터마스킹", 
    search_kwargs={"alpha": 1, "k": 1}  # Dense Vector만 사용
)
print("Dense 벡터만 사용한 검색 결과:")
print(search_results[0].page_content[:100] + "...")

Dense 벡터만 사용한 검색 결과:
<ADsP 요약정리 및 오답노트>
-1과목-
(객관식)
데이터 마스킹 : 데이터의 속성은 유지한 채, 익명으로 생성
...


### 메타데이터 필터링

메타데이터를 기준으로 검색 결과를 필터링할 수 있습니다.

**주의**: 메타데이터 필터링은 Pinecone 유료 플랜에서만 사용 가능합니다.

In [20]:
# 페이지 필터링 예시 (page < 5인 문서만 검색)
# search_results = pinecone_retriever.invoke(
#     "데이터 관련 내용을 알려줘",
#     search_kwargs={
#         "filter": {"page": {"$lt": 5}}, 
#         "k": 2
#     },
# )

# 특정 소스 문서에서만 검색
# search_results = pinecone_retriever.invoke(
#     "데이터 마스킹 관련 내용을 알려줘",
#     search_kwargs={
#         "filter": {"source": {"$eq": "ADsP.pdf"}},
#         "k": 3,
#     },
# )

## Reranking 적용 (실험적 기능)

검색 결과를 재순위화하여 더 정확한 결과를 얻을 수 있습니다.

**주의**: 이 기능은 Pinecone 라이브러리 의존성에 따라 작동하지 않을 수 있습니다.

```python
# BGE-reranker-v2-m3 모델을 사용한 재순위화
reranked_results = pinecone_retriever.invoke(
    "검색 쿼리",
    search_kwargs={
        "rerank": True, 
        "rerank_model": "bge-reranker-v2-m3", 
        "top_n": 3
    },
)
```

참고: https://docs.pinecone.io/guides/inference/rerank

## 요약

이 노트북에서는 다음 내용을 학습했습니다:

1. **Pinecone 소개**: 클라우드 기반 벡터 데이터베이스의 특징과 장단점
2. **인덱스 생성**: Pinecone 인덱스 생성 및 설정 방법
3. **문서 전처리**: PDF 문서 로드, 분할, 메타데이터 추출
4. **Sparse Encoder**: BM25 기반 Sparse Vector 생성
5. **문서 업로드**: Dense + Sparse Vector를 활용한 하이브리드 인덱싱
6. **하이브리드 검색**: PineconeKiwiHybridRetriever를 사용한 검색
7. **고급 기능**: 동적 파라미터, 메타데이터 필터링, Reranking

Pinecone의 하이브리드 검색은 Dense Vector의 의미적 유사성과 Sparse Vector의 키워드 매칭을 결합하여 더 정확한 검색 결과를 제공합니다.